In [1]:
import pandas as pd
import numpy as np
import datetime as dt

### 1- Carga de csv y análisis básico.

In [19]:
msg = pd.read_csv('datasets/messages.csv')
msg.head(4)

,CONVERSATION ID,CONVERSATION TITLE,FROM,SENDER PROFILE URL,TO,RECIPIENT PROFILE URLS,DATE,SUBJECT,CONTENT,FOLDER
0,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-08-22 21:33:15 UTC,NaN,Hola . gaston.garcia@medlog.com,INBOX
1,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,2023-08-22 13:25:29 UTC,NaN,"Gaston, te parecería que te pase esta info a a...",INBOX
2,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-08-17 15:41:07 UTC,NaN,"Perfecto, lo tengo presente por cualquier nec...",INBOX
3,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,2023-08-17 15:40:47 UTC,NaN,Hola Juan.,INBOX


In [20]:
msg.shape

(2215, 10)

In [21]:
msg.columns

Index(['CONVERSATION ID', 'CONVERSATION TITLE', 'FROM', 'SENDER PROFILE URL',
       'TO', 'RECIPIENT PROFILE URLS', 'DATE', 'SUBJECT', 'CONTENT', 'FOLDER'],
      dtype='object')

In [22]:
msg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CONVERSATION ID         2215 non-null   object 
 1   CONVERSATION TITLE      28 non-null     object 
 2   FROM                    2208 non-null   object 
 3   SENDER PROFILE URL      2179 non-null   object 
 4   TO                      2207 non-null   object 
 5   RECIPIENT PROFILE URLS  2206 non-null   object 
 6   DATE                    2215 non-null   object 
 7   SUBJECT                 0 non-null      float64
 8   CONTENT                 2178 non-null   object 
 9   FOLDER                  2215 non-null   object 
dtypes: float64(1), object(9)
memory usage: 173.2+ KB


### 2- Corrección del formato de fecha

In [23]:
# cambiamos el formato de la fecha

msg['DATE'] = pd.to_datetime(msg['DATE']) # transformamos a tipo fecha
msg['DATE'] = msg['DATE'].dt.strftime(date_format='%Y-%m-%d %H:%M:%S') # quitamos el +00:00 de la zona horaria

La cuenta de Linkedin puede tener muchos mensajes que no son de utilidad para nuestro analisis, la fecha es algo que se debera tener en cuenta. 

Esto es así porque por ejemplo, en el año 2021 la empresa ni siquiera estaba realizando esta campaña de marketing, por eso, con el objetivo de mejorar la calidad de los datos vamos a fijar una fecha para analizar los datos de ahi en adelante
- Fecha de comienzo de campaña: 2022-04-01

In [24]:
msg = msg.loc[msg['DATE']>'2022-04-01'].reset_index(drop=True)

In [25]:
len(msg) # notar que pasamos de tener 2215 registros a tener 2118

2118

### 3- Análisis del campo 'CONVERSATION TITLE'

Aquí nos dimos cuenta de que todos los registros que poseen este campo con un valor no nulo corresponden a mensajes que consideraremos spam (promociones, linkedin premium, inversiones, etc). Borraremos estos registros.


In [26]:
msg = msg[msg['CONVERSATION TITLE'].isna()]

In [27]:
len(msg) # notar que hemos descartado 21 registros mas

2097

In [28]:
msg['CONVERSATION TITLE'].isna().value_counts() # verificamos que se hayan borrado todos los registros no nulos

CONVERSATION TITLE
True    2097
Name: count, dtype: int64

### 4- Análisis del campo 'FROM'

Si recordamos la funcion info() que aplicamos en el punto 1 nos daremos cuenta que el campo 'FROM' tiene 2208/2215 datos no nulos. Los restantes los analizaremos a continuación y nos daremos cuenta que son datos perdidos. 

SI EL REGISTRO NO ESPECIFICA COMO MÍNIMO DE DONDE PROVIENE EL MENSAJE, ENTONCES ESE REGISTRO NO LE SIRVE A NUESTRO ANÁLISIS.

Habría que analizar POR QUÉ no estan estos datos.

In [29]:
msg[msg['FROM'].isna()].head(1)

,CONVERSATION ID,CONVERSATION TITLE,FROM,SENDER PROFILE URL,TO,RECIPIENT PROFILE URLS,DATE,SUBJECT,CONTENT,FOLDER
77,2-NjMxODdkODctMjIzMC00ZjdlLWIxYjAtZDQ4NDhhOTJh...,NaN,NaN,NaN,NaN,NaN,2023-08-17 12:45:53,NaN,NaN,INBOX


In [30]:
msg = msg[msg['FROM'].notna()]
len(msg) # notar que hemos descartado 7 registros mas

2090

### 5- Analizandos los campos utilizables para nuestro análisis

In [31]:
msg[msg['FROM'] == "Juan Martin Diaz"].head(2)

,CONVERSATION ID,CONVERSATION TITLE,FROM,SENDER PROFILE URL,TO,RECIPIENT PROFILE URLS,DATE,SUBJECT,CONTENT,FOLDER
1,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,2023-08-22 13:25:29,NaN,"Gaston, te parecería que te pase esta info a a...",INBOX
4,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,NaN,Juan Martin Diaz,https://www.linkedin.com/in/juan-martin-diaz,Gaston Ariel Garcia,https://www.linkedin.com/in/gaston-ariel-garci...,2023-08-17 12:44:50,NaN,"Buen día Gaston, como estas? Un gusto conectar...",INBOX


Descartaremos las columnas que no nos sirven

In [32]:
msg.drop(columns=['CONVERSATION TITLE', 'SUBJECT', 'SENDER PROFILE URL', 'RECIPIENT PROFILE URLS','FOLDER'], inplace=True)

Mensajes enviados vs Mensajes recibidos

In [33]:
enviados = len(msg[msg['FROM'] == "Juan Martin Diaz"]) ##
recibidos = len(msg[msg['TO'] == "Juan Martin Diaz"]) ##
inicio = msg['DATE'].max()
fin = msg['DATE'].min()

print(f'En el perído comprendido entre {inicio} y {fin} se han enviado {enviados} mensajes y se han recibido {recibidos} mensajes')

En el perído comprendido entre 2023-08-22 21:33:15 y 2022-04-05 13:44:13 se han enviado 1338 mensajes y se han recibido 752 mensajes


### 6- Modificación y traducción de nombres de columnas

In [34]:
msg.columns

Index(['CONVERSATION ID', 'FROM', 'TO', 'DATE', 'CONTENT'], dtype='object')

In [35]:
msg.rename(columns={'CONVERSATION ID':'CHAT_ID','FROM':'ENVIADO_POR','TO':'RECIBIDO_POR','DATE':'FECHA','CONTENT':'CONTENIDO'},inplace=True)

In [36]:
msg.head(5)

,CHAT_ID,ENVIADO_POR,RECIBIDO_POR,FECHA,CONTENIDO
0,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Gaston Ariel Garcia,Juan Martin Diaz,2023-08-22 21:33:15,Hola . gaston.garcia@medlog.com
1,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Juan Martin Diaz,Gaston Ariel Garcia,2023-08-22 13:25:29,"Gaston, te parecería que te pase esta info a a..."
2,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Gaston Ariel Garcia,Juan Martin Diaz,2023-08-17 15:41:07,"Perfecto, lo tengo presente por cualquier nec..."
3,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Gaston Ariel Garcia,Juan Martin Diaz,2023-08-17 15:40:47,Hola Juan.
4,2-MGIyMDY4NDMtYjU1YS00ODY3LTkyMjQtY2I5MTMxZWIw...,Juan Martin Diaz,Gaston Ariel Garcia,2023-08-17 12:44:50,"Buen día Gaston, como estas? Un gusto conectar..."


El siguiente paso seria crear una api con fastapi y render para mostrar en vivo un analisis de los datos.